# Menota annotated Völuspá

(AM 242 fol – Codex Wormianus v. 0.9.9) [http://clarino.uib.no/menota/document-element](http://clarino.uib.no/menota/document-element)

To use this notebook, I let you download these texts:

* [Völuspá in Codex Regius](http://clarino.uib.no/menota/texts/GKS-2365-4to-Vsp.xml)
* [Völuspá in Hauksbók](http://clarino.uib.no/menota/texts/AM-544-4to-Vsp.xml)

In [1]:
import os
os.chdir("..")

In [2]:
import os
import codecs
import html
from collections import defaultdict

import xml.etree.ElementTree as ET
from xml.sax.saxutils import escape, unescape

You can open the digitalized manuscripts here...

In [3]:
konungsbok_filename = "GKS-2365-4to-Vsp.xml"
hausbok_filename = "AM-544-4to-Vsp.xml"
menota_directory = "menota"
# #konungsbok = ET.parse(os.path.join(menota_directory, konungsbok_filename))
# # hausbok = ET.parse(os.path.join(menota_directory, hausbok_filename))
# with codecs.open(os.path.join(menota_directory, konungsbok_filename), "r", encoding="utf-8") as f:
#     konungsbok = ET.fromstring(html.unescape(f.read()))
# #    ET.parse
# #with codecs.open(os.path.join(menota_directory, hausbok_filename), "r", encoding="utf-8") as f:
# #     hausbok = ET.fromstring(html.unescape(f.read()))
    
    

... but this does not work because the namespace of tags is not defined.
The **etree** library cannot do that, whereas **lxml** can!

In [4]:
from lxml import etree

With the DTD ([Document Type Definition](https://en.wikipedia.org/wiki/Document_type_definition)) defined inside the document, you can use this text.

In [5]:
parser = etree.XMLParser(load_dtd=True, no_network=False)
tree = etree.parse(os.path.join(menota_directory, konungsbok_filename), parser=parser)

In [6]:
root = tree.getroot()
decoded_root = etree.tostring(root).decode("utf-8")
i = 170
print("\n".join(decoded_root.split("\n")[i:i+10]))

                            <me:dipl>ec</me:dipl>
                        <me:norm>ek</me:norm>
</w>
                        <w me:msa="xAJ rP gF nP cA sI" lemma="allr">
                            <me:facs>allar</me:facs>
                            <me:dipl>allar</me:dipl>
                        <me:norm>allar</me:norm>
</w>
                    </l>
                    <l>


In [7]:
print("\n".join(decoded_root.split("\n")[i:i+500]))

                            <me:dipl>ec</me:dipl>
                        <me:norm>ek</me:norm>
</w>
                        <w me:msa="xAJ rP gF nP cA sI" lemma="allr">
                            <me:facs>allar</me:facs>
                            <me:dipl>allar</me:dipl>
                        <me:norm>allar</me:norm>
</w>
                    </l>
                    <l>
                        <w me:msa="xNC gF nP cA sI" lemma="kind">
                            <me:facs>kin&#42874;ir</me:facs>
                            <me:dipl>kin&#42874;ir</me:dipl>
                        <me:norm>kindir</me:norm>
</w>
                    </l>
                    <l>
                        <w me:msa="xAJ rC gF nP cA sD" lemma="mikill">
                            <me:facs>meiri</me:facs>
                            <me:dipl>meiri</me:dipl>
                        <me:norm>meiri</me:norm>
</w>
                        <w me:msa="xCU" lemma="ok">
                            <me:facs><am>&#826

In [8]:
root.tag

'{http://www.tei-c.org/ns/1.0}TEI'

In [9]:
stringify = etree.XPath("string()")

In [10]:
namespaces = {'n': 'http://www.tei-c.org/ns/1.0',
              'me': 'http://www.menota.org/ns/1.0'}

In [11]:
header, text = root.getchildren()

In [12]:
print(text.find('.//n:div[@type="poem"]', namespaces=namespaces))
poem = text.find('.//n:div[@type="poem"]', namespaces=namespaces)

<Element {http://www.tei-c.org/ns/1.0}div at 0x1b0f3c5a6c8>


### Extraction of the poem

In [13]:
poem = text.getchildren()[0].getchildren()[3]

In [14]:
stanzas = [child for child in poem.xpath("n:lg", namespaces=namespaces)]

In [15]:
facsimile_stanzas = [stanza.findall(".//n:l", namespaces=namespaces) for stanza in stanzas]
facsimile_lines = [[line.findall(".//me:facs", namespaces=namespaces) for line in stanza] for stanza in facsimile_stanzas]
facsimile_text = [[[stringify(word) for word in line] for line in stanza ] for stanza in facsimile_lines]

In [16]:
diplomatica_stanzas = [stanza.findall(".//n:l", namespaces=namespaces) for stanza in stanzas]
diplomatica_lines = [[line.findall(".//me:dipl", namespaces=namespaces) for line in stanza] for stanza in diplomatica_stanzas]
diplomatica_text = [[[stringify(word) for word in line] for line in stanza ] for stanza in diplomatica_lines]

In [17]:
normalized_stanzas = [stanza.findall(".//n:l", namespaces=namespaces) for stanza in stanzas]
normalized_lines = [[line.findall(".//me:norm", namespaces=namespaces) for line in stanza] for stanza in normalized_stanzas]
normalized_text = [[[stringify(word) for word in line] for line in stanza ] for stanza in normalized_lines]

In [18]:
lemmata_stanzas = [stanza.findall(".//n:l", namespaces=namespaces) for stanza in stanzas]
lemmata_lines = [[line.findall(".//n:w", namespaces=namespaces) for line in stanza] for stanza in lemmata_stanzas]
lemmata_text = [[[word.get("lemma") for word in line] for line in stanza ] for stanza in lemmata_lines]

In [19]:
pos_stanzas = [stanza.findall(".//n:l", namespaces=namespaces) for stanza in stanzas]
pos_lines = [[line.findall(".//n:w", namespaces=namespaces) for line in stanza] for stanza in pos_stanzas]
pos_text = [[[word.get('{http://www.menota.org/ns/1.0}msa') for word in line] for line in stanza ] for stanza in pos_lines]

#### First stanza

In [20]:
facsimile_text[0]

[['Hlıoðſ', 'bið', 'ec', 'allar'],
 ['kinꝺir'],
 ['meiri', '⁊', 'miɴi'],
 ['maugo', 'heimꝺalar'],
 ['uilðo', 'at', 'ec', 'ualꝼꜹþ̅'],
 ['uel', 'ꝼyr', 'telia'],
 ['ꝼoꝛn', 'ſpioll', 'ꝼíra'],
 ['þꜹ', 'e͛', 'ꝼremſt', 'u̅', 'man', '.']]

In [21]:
diplomatica_text[0]

[['Hlıoðſ', 'bið', 'ec', 'allar'],
 ['kinꝺir'],
 ['meiri', 'oc', 'miɴi'],
 ['maugo', 'heimꝺalar'],
 ['uilðo', 'at', 'ec', 'ualꝼꜹþr'],
 ['uel', 'ꝼyr', 'telia'],
 ['ꝼoꝛn', 'ſpioll', 'ꝼíra'],
 ['þꜹ', 'er', 'ꝼremſt', 'um', 'man', '.']]

In [22]:
normalized_text[1]

[['Ek', 'man', 'jötna'],
 ['ár', 'um', 'borna'],
 ['þá', 'er', 'forðum', 'mik'],
 ['fædda', 'höfðu', '.'],
 ['Níu', 'man', 'ek', 'heima'],
 ['níu', 'íviðjur'],
 ['mjötvið', 'mæran'],
 ['fyr', 'mold', 'neðan', '.']]

In [23]:
lemmata_text[1]

[['ek', 'muna', 'iǫtunn'],
 ['ár', 'um', 'bera'],
 ['sá', 'er', 'forðum', 'ek'],
 ['fǿða', 'hafa'],
 ['níu', 'muna', 'ek', 'heimr'],
 ['níu', None],
 ['mjǫtviðr', 'mǽrr'],
 ['fyr', 'mold', 'neðan']]

In [24]:
print(pos_text[0])

[['xNC gN nS cG sI', 'xVB fF tPS mIN p1 nS vA iST', 'xPD cN', 'xAJ rP gF nP cA sI'], ['xNC gF nP cA sI'], ['xAJ rC gF nP cA sD', 'xCU', 'xAJ rC gF nP cA sD'], ['xNC gM nP cA sI', 'xNP gM cG sI'], ['xVB fF tPS mIN p2 nS vA iWK', 'xCU', 'xPD cN', 'xNC gM nS cN'], ['xAV rP', 'xAV rP', 'xVB fF tPS mSU p1 nS vA iWK'], ['xAJ rP gN nP cA sI', 'xNC gN nP cA sI', 'xNC gM nP cG sI'], ['xPD gN nP cA', 'xCU', 'xAJ rS gN nP cA sI', 'xAV rP', 'xVB fF tPS mIN p1 nS vA iPP']]


### Parsing the POS tags

These annotations are ununderstandable without the docs that you can find [here](https://menota.org/HB3_ch11.xml#sec11.3). The idea is to parse these annotations the same way I did in https://github.com/cltk/old_norse_texts_heimskringla/blob/master/pos.py

In [25]:
from pos_menota import *

In [26]:
c = POSMenota.parse("xVB fP tPT vA gM nS cN sI", True)
print(c.vectorize())

[True, False, False, True, False, False, True, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, True, True, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False]
